In [1]:
import os # Handle OS file systems and directory paths
import random # Random number
import numpy as np # Import NumPy for arrays and such
import matplotlib.pyplot as plt # Plotting Tools
import numpy.linalg as npl # Linear Algebra Library
import time # Time
# from google.colab import drive # Read files in from Google Drive
import pandas as pd # Import Pandas for data processing
import datetime as dt
import scipy.io as sio
import io
from decimal import Decimal # String to double conversion
from glob import glob  # 从 glob 模块中导入 glob 函数

Matplotlib is building the font cache; this may take a moment.


In [2]:
import boto3
from botocore.exceptions import NoCredentialsError

In [8]:
# Optionally, you can specify the AWS region
s3 = boto3.client('s3', region_name='us-east-1')  # Replace 'us-west-2' with your AWS region

import pandas as pd
# s3_uri = 's3://pepper-dataset/crop_data/crop_IOWA/Iowa_corn_2019.csv'
# df = pd.read_csv(s3_uri)

#Read files from s3
bucket_name = 'pepper-dataset'
prefix = 'Temp/processed_with_all_features_ver_2/'

In [9]:
# List to store all object keys
county_files = []

# Initial call to list_objects_v2
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# Process the initial response
while True:
    # Collect keys from current response
    current_keys = [obj['Key'] for obj in response.get('Contents', [])]
    county_files.extend(current_keys)

    # Check if there are more objects to retrieve
    if not response['IsTruncated']:
        break  # No more objects to retrieve

    # Get ContinuationToken for next paginated call
    continuation_token = response['NextContinuationToken']

    # Make subsequent call with ContinuationToken
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, ContinuationToken=continuation_token)
    
    
county_files = ["/" + path if not path.startswith("/") else path for path in county_files]

In [10]:
len(county_files)

11407

In [15]:
# try 1 county
year_range = [2002, 2022]
years = [x for x in range(year_range[0], year_range[-1]+1)]

filtered_files = [file_path for file_path in county_files
                  if any(str(year) in file_path for year in years)]


print(len(filtered_files))


filtered_files[:50]


df = pd.read_csv('s3://'+bucket_name+filtered_files[0])

10399


In [26]:
import pandas as pd

summary_list = []

# 要忽略的列
skip_columns = ['sshf', 'evavt']

for i in range(len(filtered_files)):
    file_path = 's3://' + bucket_name + filtered_files[i]
    df = pd.read_csv(file_path)

    file_summary = {'file': filtered_files[i]}

    for column in df.columns:
        if column in skip_columns:
            continue
        if pd.api.types.is_numeric_dtype(df[column]):
            min_val = df[column].min()
            max_val = df[column].max()
            file_summary[f"{column}_min"] = min_val
            file_summary[f"{column}_max"] = max_val
        # 非数值列可选跳过或记录
    summary_list.append(file_summary)

summary_df = pd.DataFrame(summary_list)


summary_df.to_csv('s3://'+ bucket_name+ '/check_iowa.csv', index=False)

print("exported to check_iowa.csv")


exported to check_iowa.csv
